In [5]:
import pandas as pd
import numpy as np
#import datetime
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## Funzioni accessorie

In [11]:
def moving_function(data, fun, left_win, right_win):
    '''Applica una funzione mobile su un array rispetto alle ampiezze indicate
    per le finestre a sx e dx
    '''
    out = np.zeros_like(data)
    num_samples = len(data)
    lefts = list(map(lambda x: x-left_win, list(range(num_samples))))
    rights = list(map(lambda x: x+right_win+1, list(range(num_samples))))
    for i in range(num_samples):
        out[i] = fun(data[max(0, lefts[i]):min(num_samples, rights[i])])
    return out


## Parametri

In [10]:
data_file = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'


lisciatura=3

## Lettura dati e preprocessing

In [7]:
data=pd.read_csv(data_file)

giorni=pd.unique(data['data'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(data['denominazione_regione'])

for colname in data.columns:
    print(colname,end=', ')


Giorni osservati: 688
Da: 2020-02-24
 A: 2022-01-11
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2, 

### Formazione dataframe nazionale

In [9]:
data_italia = pd.DataFrame(data.groupby(['data']).sum())

data_italia.tail(5)


,codice_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
data,,,,,,,,,,,,,,,,,,,,,
2022-01-07T17:00:00,249,903.972147,256.745065,14591,1499,16090,1657981,1674071,80492,108304,...,0.0,0.0,7083762,145945664,42287504.0,120.0,6080761.0,1003001.0,75250881.0,70694783.0
2022-01-08T17:00:00,249,903.972147,256.745065,14930,1557,16487,1802406,1818893,144822,197552,...,0.0,0.0,7281297,147165930,42520576.0,154.0,6164375.0,1116922.0,75568209.0,71597721.0
2022-01-09T17:00:00,249,903.972147,256.745065,15647,1595,17242,1926737,1943979,125086,155659,...,0.0,0.0,7436939,148159131,42725013.0,142.0,6229650.0,1207289.0,75856287.0,72302844.0
2022-01-10T17:00:00,249,903.972147,256.745065,16340,1606,17946,1986651,2004597,60618,101762,...,0.0,0.0,7554344,148771952,42856913.0,114.0,6286591.0,1267753.0,76085265.0,72686687.0
2022-01-11T17:00:00,249,903.972147,256.745065,17067,1677,18744,2115395,2134139,129542,220532,...,0.0,0.0,7774863,150147466,43105796.0,185.0,6361711.0,1413152.0,76403257.0,73744209.0


## Grafici nazionali

In [ ]:
indicatori_selected = ['totale_casi',
                       'nuovi_positivi',
                       'guariti_giornalieri',
                       'variazione_totale_positivi',
                       'totale_positivi',
                       'deceduti_giornalieri']


In [ ]:
for indicatore in indicatori_selected:
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data_italia['data'],
                             y=data_italia[indicatore],
                             mode='markers'))
    # versione lisciata
    fig.add_trace(go.Scatter(x=data_italia['data'],
                             y=moving_function(data_italia[indicatore],np.mean,lisciatura,lisciatura),
                             mode='lines'))

    fig.update_layout(
        title='%s, Italia. Odierno %d' % (
            indicatore, data_italia[indicatore]['Italia'][-1]),
        autosize=False,
        showlegend=False,
        width=900,
        height=600)
    fig.show()
